# ESTRATÉGIA S-RANK

- Estratégia de seleção de fundos imobiliários

- Passo 1 Aplicação de Filtros:
 1. Filtro de liquidez diária de movimentação.
 2. Filtro de Tipo de fundos imobiliários.
 3. Filtro de Estabilidade.
 4. Filtro de Idade.

In [1]:
import sqlite3
import pandas as pd 
import os

# 1 . Filtro de Liquidez

- Realizamos um filtro para verificar os fundos com menos de 200 mil reais de negociação diaria

In [13]:
#Conexão com o database
conn = sqlite3.connect(r'database_fii.db')

In [14]:
#criação do cursor para vermos as tabelas do banco
cur = conn.cursor()
cur.execute("""select name from sqlite_master where type='table';""")
cur.fetchall()

[('ranking_diario',), ('ipo_data_list',), ('tipo_do_fundo',)]

In [15]:
#selecionar a tabela ranking_diário para podermos aplicar a estratégia s-rank
df = pd.read_sql_query("""SELECT * FROM ranking_diario""",conn)
df.head()

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) Média,...,VPA,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. no Período,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos,Hoje
0,FIVN11,Shoppings,4.00,1570,0.00,0.00,0.00,0.00,0.00,0.00,...,7.60,0.53,,,,,56.00,,1,2020-10-21 13:50:11.132171
1,BZLI11,Títulos e Val Mob,14.90,6123,0.00,0.00,0.00,0.00,0.00,0.00,...,11.86,1.26,,,,,,,0,2020-10-21 13:50:11.132171
2,XTED11,Lajes Corporativas,7.32,3950,0.00,0.00,0.00,0.00,0.00,0.00,...,14.52,0.50,,,,,100.00,100.00,1,2020-10-21 13:50:11.132171
3,ALMI11,Lajes Corporativas,1224.67,160,0.00,0.00,0.00,0.00,0.00,0.00,...,1994.57,0.61,,,,,71.19,,1,2020-10-21 13:50:11.132171
4,DOMC11,Lajes Corporativas,439.00,42,0.00,0.00,0.00,0.00,0.00,0.00,...,162.97,0.41,,,,,23.44,,1,2020-10-21 13:50:11.132171


In [16]:
#ver os tipos das colunas do df
df.dtypes

Código do fundo             object
Setor                       object
Preço Atual                 object
Liquidez Diária             object
Dividendo                   object
Dividend Yield              object
DY (3M) Acumulado           object
DY (6M) Acumulado           object
DY (12M) Acumulado          object
DY (3M) Média               object
DY (6M) Média               object
DY (12M) Média              object
DY Ano                      object
Variação Preço              object
Rentab. Período             object
Rentab. Acumulada           object
Patrimônio Líq.             object
VPA                         object
P/VPA                       object
DY Patrimonial              object
Variação Patrimonial        object
Rentab. Patr. no Período    object
Rentab. Patr. Acumulada     object
Vacância Física             object
Vacância Financeira         object
Quantidade Ativos           object
Hoje                        object
dtype: object

In [17]:
#criar função para converter as colunas em tipos númericos
def coerce_columns_to_numeric(df,column_list):
    df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')

In [18]:
#converter a coluna Liquidez Diária para numeric
coerce_columns_to_numeric(df, 'Liquidez Diária')
coerce_columns_to_numeric(df, 'Dividend Yield')
coerce_columns_to_numeric(df, 'DY (12M) Média')

In [19]:
#removemos os FIIS com menos de 200 mil reais em liquidez diária dos ultimos doze meses
rank = df.loc[df['Liquidez Diária'] >= 20000]

# 2. Filtro de Tipos de Fundos

- Remove os fundos de incorporação ou fundos de desenvolvimento

In [21]:
type_funds = pd.read_sql_query('SELECT * FROM tipo_do_fundo',conn)

In [22]:
type_funds.columns

Index(['Código do fundo', 'Público Alvo', 'Tipo de FII'], dtype='object')

In [23]:
type_funds = type_funds.rename(columns={'Ticker':'Código do fundo'})


In [24]:
type_funds.drop(type_funds[type_funds['Tipo de FII'] == 'Tijolo: Desenvolvimento'].index, inplace=True)

In [25]:
rank_2 = pd.merge(rank,type_funds, on='Código do fundo')

# 3. Criar um filtro de estabilidade
- o filtro de estabilidade é a diferença entre a média e a mediana do DY (dividend Yield)

In [26]:
#pegar a mediana do Dividend Yield anual
rank_2['DY mediana'] = round(abs(rank_2['Dividend Yield'] - rank_2['Dividend Yield']/12),2)

In [27]:
rank_2['Dif entre med e media'] = rank_2['DY mediana'] - rank_2['DY (12M) Média']

In [28]:
rank_2['filtro estabilidade'] = rank_2['Dif entre med e media'].apply(lambda x: 'Instavel' if x < 0
else 'Estavel' if x > 0
else '')

In [29]:
rank_2.head()

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) Média,...,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos,Hoje,Público Alvo,Tipo de FII,DY mediana,Dif entre med e media,filtro estabilidade
0,GSFI11,Shoppings,2.60,113089.0,0.00,0.03,0.03,0.03,0.03,0.01,...,,,,10,2020-10-21 13:50:11.132171,Geral,Tijolo: Shoppings,0.03,0.03,Estavel
1,GSFI11,Shoppings,2.60,113089.0,0.00,0.03,0.03,0.03,0.03,0.01,...,,,,10,2020-10-21 13:50:11.132171,None,None,0.03,0.03,Estavel
2,HGRE11,Lajes Corporativas,150.75,41682.0,0.65,0.43,1.34,2.50,5.13,0.45,...,3.72,26.51,28.52,22,2020-10-21 13:50:11.132171,Geral,Tijolo: Escritórios,0.39,-0.04,Instavel
3,HGRE11,Lajes Corporativas,150.75,41682.0,0.65,0.43,1.34,2.50,5.13,0.45,...,3.72,26.51,28.52,22,2020-10-21 13:50:11.132171,None,None,0.39,-0.04,Instavel
4,HGRE11,Lajes Corporativas,150.00,25195.0,0.97,0.62,1.48,2.82,5.20,0.49,...,,22.41,25.04,21,2021-01-26 12:07:02.764778,Geral,Tijolo: Escritórios,0.57,0.14,Estavel


# 4. Filtro de Idade
- Retira fundos que tem menos de 1 ano de existência.

In [75]:
ipo_list = pd.read_sql_query('SELECT * FROM ipo_data_list',conn)

In [76]:
ipo_list.columns

Index(['Nome ', 'Código do fundo', 'Data_Ipo', 'Valor Ipo', 'Segmento',
       'Administrador '],
      dtype='object')

In [88]:
nr = pd.merge(rank_2,ipo_list, on='Código do fundo')

In [89]:
nr.columns

Index(['Código do fundo', 'Setor', 'Preço Atual', 'Liquidez Diária',
       'Dividendo', 'Dividend Yield', 'DY (3M) Acumulado', 'DY (6M) Acumulado',
       'DY (12M) Acumulado', 'DY (3M) Média', 'DY (6M) Média',
       'DY (12M) Média', 'DY Ano', 'Variação Preço', 'Rentab. Período',
       'Rentab. Acumulada', 'Patrimônio Líq.', 'VPA', 'P/VPA',
       'DY Patrimonial', 'Variação Patrimonial', 'Rentab. Patr. no Período',
       'Rentab. Patr. Acumulada', 'Vacância Física', 'Vacância Financeira',
       'Quantidade Ativos', 'Hoje', 'Público Alvo', 'Tipo de FII',
       'DY mediana', 'Dif entre med e media', 'filtro estabilidade', 'Nome ',
       'Data_Ipo', 'Valor Ipo', 'Segmento', 'Administrador '],
      dtype='object')

In [90]:
import datetime

In [91]:
today = datetime.date.today()

In [92]:
d = today.day
m = today.month
y = today.year

In [93]:
l_y = y-1

In [94]:
data = '{}/0{}/{}'.format(d,m,l_y)

In [95]:
data

'22/02/2020'

In [97]:
nr = nr.loc[nr['Data_Ipo'] < data]

In [99]:
nr.drop_duplicates('Código do fundo',inplace=True)

In [100]:
nr.to_excel(r's_rank{}.xlsx'.format(str(today)),index=False)